## Week 2 Day 3

Now we get to more detail:

1. Different models

2. Structured Outputs

3. Guardrails

In [40]:
from dotenv import load_dotenv
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel, input_guardrail, GuardrailFunctionOutput
from typing import Dict
import sendgrid
import os
import asyncio
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel, Field

In [8]:
load_dotenv(override=True)
myGmail = os.environ.get('MY_GMAIL')

In [4]:
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Google API Key not set (and this is optional)
DeepSeek API Key not set (and this is optional)
Groq API Key not set (and this is optional)


In [13]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

### It's easy to use any models with OpenAI compatible endpoints

In [6]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
DEEPSEEK_BASE_URL = "https://api.deepseek.com/v1"
GROQ_BASE_URL = "https://api.groq.com/openai/v1"

In [7]:

deepseek_client = AsyncOpenAI(base_url=DEEPSEEK_BASE_URL, api_key=deepseek_api_key)
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
groq_client = AsyncOpenAI(base_url=GROQ_BASE_URL, api_key=groq_api_key)

deepseek_model = OpenAIChatCompletionsModel(model="deepseek-chat", openai_client=deepseek_client)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)
llama3_3_model = OpenAIChatCompletionsModel(model="llama-3.3-70b-versatile", openai_client=groq_client)

In [25]:
sales_agent1 = Agent(name="DeepSeek Sales Agent", instructions=instructions1, model="gpt-4o-mini")
sales_agent2 =  Agent(name="Gemini Sales Agent", instructions=instructions2, model="gpt-4o-mini")
sales_agent3  = Agent(name="Llama3.3 Sales Agent",instructions=instructions3,model="gpt-4o-mini")

In [26]:
description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

In [27]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(myGmail)  # Change to your verified sender
    to_email = To(myGmail)  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [28]:
subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [29]:
email_tools = [subject_tool, html_tool, send_html_email]

In [30]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [31]:
tools = [tool1, tool2, tool3]
handoffs = [emailer_agent]

In [32]:
sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""



In [33]:
sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Automated SDR"):
    result = await Runner.run(sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

In [10]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [11]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)

In [ ]:
careful_sales_manager = Agent(
    name="Sales Manager",
    instructions=sales_manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a cold sales email addressed to Dear CEO from Alice"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

## Check out the trace:

https://platform.openai.com/traces

In [ ]:

message = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

## Using Structured Outputs for Email Generation

Let's improve our email generation by using structured outputs. This makes it easier to:
- Extract subject and body separately
- Validate email structure
- Work with email data programmatically


In [ ]:
# Define a Pydantic model for structured email output
# Field is already imported in cell 1, but we'll ensure it's available here
from pydantic import Field  # Import Field if not already available

class ColdSalesEmail(BaseModel):
    """A structured cold sales email with subject and body"""
    subject: str = Field(description="The email subject line that is likely to get a response")
    body: str = Field(description="The email body text, including greeting and closing")
    tone: str = Field(description="The tone of the email (e.g., professional, engaging, concise)")
    
    def to_markdown(self) -> str:
        """Convert the email to markdown format"""
        return f"Subject: {self.subject}\n\n{self.body}"
    
    def __str__(self) -> str:
        """String representation of the email"""
        return self.to_markdown()


In [39]:
# Create sales agents with structured outputs
structured_sales_agent1 = Agent(
    name="Professional Sales Agent",
    instructions=instructions1,
    model="gpt-4o-mini",
    output_type=ColdSalesEmail  # This enforces structured output!
)

structured_sales_agent2 = Agent(
    name="Engaging Sales Agent",
    instructions=instructions2,
    model="gpt-4o-mini",
    output_type=ColdSalesEmail
)

structured_sales_agent3 = Agent(
    name="Busy Sales Agent",
    instructions=instructions3,
    model="gpt-4o-mini",
    output_type=ColdSalesEmail
)


### Benefits of Structured Outputs:

1. **Type Safety**: We get a typed object instead of a string
2. **Easy Access**: Direct access to `email.subject`, `email.body`, etc.
3. **Validation**: Pydantic automatically validates the structure
4. **Better Tool Integration**: Easier to pass structured data to other tools


In [41]:
# Parallel generation with structured outputs
message = "Write a cold sales email addressed to 'Dear CEO'"

with trace("Parallel Structured Emails"):
    results = await asyncio.gather(
        Runner.run(structured_sales_agent1, message),
        Runner.run(structured_sales_agent2, message),
        Runner.run(structured_sales_agent3, message),
    )

# Now we have structured email objects!
emails = [result.final_output for result in results]

# Easy to work with structured data
for i, email in enumerate(emails, 1):
    print(f"\n=== Email {i} ({email.tone}) ===")
    print(f"Subject: {email.subject}")
    print(f"Body length: {len(email.body)} characters")
    print()



=== Email 1 (professional) ===
Subject: Enhance Your SOC2 Compliance Effortlessly with ComplAI
Body length: 920 characters


=== Email 2 (humorous and engaging) ===
Subject: Let’s Make SOC2 Compliance Less Painful (and Maybe a Little Fun)
Body length: 1067 characters


=== Email 3 (professional) ===
Subject: Ensure SOC2 Compliance Effortlessly with ComplAI
Body length: 620 characters



In [42]:
# Update send_email function to work with structured outputs
# Note: Tools work best with simple types, so we accept subject and body separately
# When agents with structured outputs are converted to tools, their outputs are automatically handled
@function_tool
def send_structured_email(subject: str, body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(myGmail)
    to_email = To(myGmail)
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success", "subject": subject}

# Convert structured agents to tools
structured_tool1 = structured_sales_agent1.as_tool(
    tool_name="structured_sales_agent1", 
    tool_description="Write a cold sales email (returns structured output)"
)
structured_tool2 = structured_sales_agent2.as_tool(
    tool_name="structured_sales_agent2", 
    tool_description="Write a cold sales email (returns structured output)"
)
structured_tool3 = structured_sales_agent3.as_tool(
    tool_name="structured_sales_agent3", 
    tool_description="Write a cold sales email (returns structured output)"
)

structured_tools = [structured_tool1, structured_tool2, structured_tool3, send_structured_email]


### Note: Working with Structured Outputs in Tools

When agents are converted to tools using `as_tool()`, the structured output is automatically handled. The tool receives the Pydantic model instance, making it easy to access fields like `email.subject` and `email.body`.

**Key Advantage**: Instead of parsing text to extract subject and body, we get validated structured objects!


In [49]:
# Example: Using structured outputs in a complete workflow
# When agents with structured outputs are used as tools, the calling agent
# receives the structured data and can work with subject/body directly

structured_sales_manager_instructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three structured_sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: The structured_sales_agent tools return structured emails with 'subject' and 'body' fields. Pass ONLY subject and body of the winning email draft to send_structured_email. send_structured_email will take care of formatting and sending.
 

Crucial Rules:
- You must use the structured sales agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to send_structured_email — never more than one.
"""

structured_sales_manager = Agent(
    name="Structured Sales Manager",
    instructions=structured_sales_manager_instructions,
    tools=structured_tools,
    model="gpt-4o-mini"
)

# Note: When this runs, the structured outputs from the sales agents will be
# automatically available to the sales manager, making it easy to extract
# subject and body fields!


In [50]:

message = "Send out a cold sales email addressed to Dear CEO"

with trace("Structured Automated SDR"):
    result = await Runner.run(structured_sales_manager, message)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Try different models<br/>• Add more input and output guardrails<br/>• Use structured outputs for the email generation
            </span>
        </td>
    </tr>
</table>